# Import

In [1]:
import json
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import logging
import os
import tqdm
import torch
import math
import numpy as np
from tqdm import tqdm_notebook
from typing import List

In [2]:
BOS = '<s>'
EOS = '</s>'
MASK = '<mask>'
PAD = '<pad>'

tokenizer = PreTrainedTokenizerFast(tokenizer_file='Tokenizer/tokenizer.json',
            bos_token=BOS, eos_token=EOS, unk_token='<unk>', 
            pad_token=PAD, mask_token=MASK)

In [3]:
class HistoryDataset(Dataset):
    def __init__(self, historyData,tokenizer):
        self.data = historyData
        self.tokenizer = tokenizer
        self.max_sequence_length = 512
        self.sos_token = tokenizer.convert_tokens_to_ids("<s>")
        self.eos_token = tokenizer.convert_tokens_to_ids("</s>")
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        item = self.data[index]
        context_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(f"{item}"))
        conditional_tokens_len = 1 + len(context_tokens) + 1
        
        if conditional_tokens_len > self.max_sequence_length:
            available_seq_len = (
                self.max_sequence_length - conditional_tokens_len + len(context_tokens)
            )
            context_tokens = context_tokens[:available_seq_len]
            
            
        tokens = [self.sos_token] + context_tokens + [self.eos_token]
        
        assert len(tokens) <= self.max_sequence_length
        #tokens = tokens + [3] * (self.max_sequence_length - len(tokens))
        return torch.tensor(tokens)

In [4]:
class TqdmLoggingHandler(logging.Handler):
    def __init__(self, level=logging.NOTSET):
        super().__init__(level)

    def emit(self, record):
        try:
            msg = self.format(record)
            tqdm.tqdm.write(msg)
            self.flush()
        except (KeyboardInterrupt, SystemExit):
            raise
        except Exception:
            self.handleError(record)

In [5]:
def _create_logger(output_dir: str):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(asctime)s] %(message)s")

    file_handler = logging.FileHandler(os.path.join(output_dir, "train.log"))
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    handler = TqdmLoggingHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s"))
    logger.addHandler(handler)
    return logger

In [ ]:
doc_list = []
with open('data/as_set.json') as f:
    data = json.load(f)

for i in data['data']:
    doc_list.append(i['doc'])
doc_list = list(set(doc_list))

logger = _create_logger(output_dir="hisotry_finetuning_outputs")

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

history_dataset = HistoryDataset(doc_list, tokenizer)
history_data_loader = DataLoader(history_dataset, batch_size=1, shuffle=True, pin_memory=True, collate_fn = lambda x :x)

learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
logging.info('KoGPT-2 Transfer Learning Start')

epochs=200
loss_list_between_log_interval = []
count = 0
for epoch in range(epochs):

    for data in tqdm_notebook(history_data_loader):
        data = data[0]
        optimizer.zero_grad()
        data= data.to(device)
    
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        
        loss_list_between_log_interval.append(outputs.loss.item())
        
        if count %100 ==0:
            mean_loss = np.mean(loss_list_between_log_interval)
            logging.info('epoch no.{} train no.{}  loss = {} perplexity = {}' . format(epoch, count+1, loss, math.exp(mean_loss)))

        if (count >0 and count%1000==0):
            state_dict = model.state_dict()
            model_path = os.path.join("hisotry_finetuning_outputs", f"gpt2_step_{count}.pth")
            torch.save(state_dict, model_path)
        count += 1

# 문장 생성

In [10]:
import torch
from transformers import GPT2LMHeadModel

text = '가군은 장안성 축성을 담당한 관리로'

#  Load KoGPT2 model
kogpt_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
input_ids = tokenizer.encode(text)
gen_ids = kogpt_model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
kogpt2_generated = tokenizer.decode(gen_ids[0,:].tolist())

print("KoGPT2")
print(kogpt2_generated)
print()

# Load After History Finetuning model
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.load_state_dict(torch.load('hisotry_finetuning_outputs/gpt2_step_200000.pth', map_location="cpu"))
tokenizer = PreTrainedTokenizerFast(tokenizer_file='Tokenizer/tokenizer.json',
            bos_token=BOS, eos_token=EOS, unk_token='<unk>', 
            pad_token=PAD, mask_token=MASK)
input_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())

print("After History Fintuning: ")
print(generated)
print()


KoGPT2
가군은 장안성 축성을 담당한 관리로 알려져 있다.
이번 전시는 지난해부터 추진해온 '장안읍성과 함께하는 장안의 역사문화유산' 사업의 일환으로 마련됐다.
전시에는 ▲조선시대와 조선시대를 대표하는 인물들의 초상화와 사진, 영상 등 다양한 볼거리들이 선보인다.
특히 조선 후기 문신인 이강(李康)의 초상화도 만날 수 있어 눈길을 끈다.
또한 이번 전시회에서는 조선후기 문인화가인 김학(金鶴) 화백의 작품 30여점이 소개된다.
김화백은 "중국의 문인들이 남긴 유품들을 통해 우리 민족의 얼과 정신을 느낄수 있는 좋은 기회가 될 것"이라고 말했다.</d> 서울시는 오는

After History Fintuning: 
가군은 장안성 축성을 담당한 관리로 관직은 참군이었다. 조선 건국 초기의 명신으로서 이근행이 있었다.</s>

